<a id='top'></a>
---

# Tri-Vibe Sensor Integration Notebook
* Language: Python 3.7  
* Required [Libraries](#libraries)
* Author: Leo Bach 12/18/2019
* Version: 5.01  
    (Object Oriented Design Implemented)  
    (CSV Storage Added)  
    (Charting Added)  
    (HTML Storage Added)  
* Email: leo@machinesaver.net  
* Phone: +1(832)581-9908

### Table of Contents:
[Part 1: Required Libraries for TRIVIBE CLASS](#libraries)  
[Part 2: TRIVIBE CLASS](#trivibeclass)  
[Part 3: Required Libraries for NOTEBOOK](#notebooklibraries)  
[Part 4: Find Computer's RS485 Port](#checkport)  
[Part 5: Available TRIVIBE Methods (Functions)](#methodlist)  
[Part 6: Create a Sensor Object from TRIVIBE CLASS](#instantiation)  
[Part 7: Available TRIVIBE Attributes (Properties)](#attributelist)  
[Part 8: Example Static Attributes - Sensor Info](#staticattributes)  
[Part 9: Example Uptime](#exampleuptime)  
[Part 10: Example Temperature](#exampletemperature)  
[Part 11: Example Vibration](#examplevibration)  
[Part 12: Example Easy Snapshot](#easysnapshot)  
[Part 13: Example Snapshot Storage (CSV)](#storecsv)  
[Part 14: Example Snapshot Chart (Plotly)](#chartcsv)  
[Part 15: Example Snapshot Chart Storage (HTML)](#storehtml)  
[Part 16: Example Advanced Snapshot](#advancedsnapshot)  

<a id='libraries'></a>
---

# Part 1: Required Libraries for [TRIVIBE CLASS](#trivibeclass)  

In [1]:
# import required libraries for TRIVIBE CLASS
import minimalmodbus                        # modbus master library which supports Modbus RTU & Modbus ASCII but not Modbus TCP
import time                                 # use sleep to wait while sensor processes are happening + used to take timestamp when data is collected
from datetime import datetime               # manipulate timestamp to human readable date
import numpy as np                          # number manipulation - calculate snapshot reads, create time arrays (twf - x-axis) and frequency bins arrays (spectrum - x-axis)
from scipy.fftpack import fft               # fast fourier transform algorithm
from pathlib import Path
from openpyxl import Workbook
import plotly.graph_objects as go           # charting library
import xlsxwriter

# remove libraries?
import os                                   # used to save snapshots to /snapshots directory within Machine Saver's Jupyter notebook
import matplotlib.pyplot as plt             # charting library
import pandas as pd                         # used to manipulate numbers
import os                                   # used to save snapshots to /snapshots directory within Machine Saver's Jupyter notebook
import math                                 # used to manipulate numbers

<a id='trivibeclass'></a>
---

# Part 2: TRIVIBE CLASS

In [2]:
class TRIVIBE:
    manufacturer = 'Machine Saver Inc.'
    
    def __init__(self, slaveAddress, serialPort):
        self.serialPort = serialPort
        self.modbus = minimalmodbus.Instrument(self.serialPort, slaveAddress)
        self._setSerialPortSettings()
        
        self.info = {
            "serial": self._getSerialNumber(),
            "revision": self._getRevision(),
            "board_type": self._getBoardType(),
            "internal_accelerometer_1":{},
            "internal_accelerometer_2":{}
        }
        self._getSensitivity()
        
        self.uptime ={}
        
        self.vibration = {}

        self.temperature = {}

        # TODO: add alarms
        self.alarms = {}
#         "channel_1":{},
#         "channel_2":{},
#         "channel_3":{},
#         "channel_4":{},
        
        # TODO: add snapshot
        self.snapshot = {}
#         "engine_status":"idle",
#         "samples_per_axis":0,
#         "capture_time":0,
#         "triaxial_RAW":[],
#         "axis_1_RAW":[],
#         "axis_2_RAW":[],
#         "axis_3_RAW":[],
#         "timewaveform_time_ms":[],
#         "axis_1_acceleration":[],
#         "axis_2_acceleration":[],
#         "axis_3_acceleration":[],
#         "spectrum_frequency_hz":[],
#         "axis_1_spectrum":[],
#         "axis_2_spectrum":[],
#         "axis_3_spectrum":[]
        
    def _setSerialPortSettings(self):
        '''Update serial communication settings to talk to Tri-Vibe'''
        self.modbus.serial.baudrate = 115200           # Baudrate fixed 115200
        self.modbus.serial.bytesize = 8                # Bytesize fixed 8
        self.modbus.serial.parity   = "N"              # Parity fixed None 
        self.modbus.serial.stopbits = 1                # Stopbits fixed 1
        self.modbus.serial.timeout  = 0.10             # Seconds
        self.modbus.close_port_after_each_call = False  # Helps communication for Windows Devices (can be set to False on many Linux devices)
        self.modbus.mode = minimalmodbus.MODE_RTU      # modbus mode fixed RTU Mode
        self.modbus.clear_buffers_before_each_transaction = True
        time.sleep(0.1)
    
    def _getRevision(self):
        '''Reads Modbus Register 0. Scales return value to Revision. Updates Trivibe Object Property info["revision"].'''
        sensor_revision = self.modbus.read_register(0, functioncode=3)  # Revision at time of writing 112
        return(sensor_revision-768)                                  # Remove the 768 from the output of register 0 to get the sensor's revision


    def _getSerialNumber(self):
        '''Reads Modbus Register 26. Updates Trivibe Object Property info["serial"].'''
        serial = self.modbus.read_long(26, functioncode=3)
        return serial
        
    def _modbus_map_board_type(self, x): # checks the board type
        return {
            0:['Unset','Unset'],
            1:['2g','2g'],
            2:['2g','4g'],
            3:['4g','2g'],
            4:['2g','25g'],
            5:['4g','25g'],
            6:['25g','2g'],
            7:['25g','4g'],
            8:['4g','10g'],
            9:['10g','4g'],
            10:['100g','10g'],
        }.get(x, 'error')    # 'error' is default if x not found


    def _getBoardType(self):
        '''Reads Modbus Register 366. Updates Trivibe Object Property info[board_type].'''
        return(self._modbus_map_board_type(self.modbus.read_register(366, functioncode=3)))
    
    def _frequency_map(self, x): # checks the board type
        return {
            '2g':1000,
            '4g':1000,
            '10g':5000,
            '20g':5000,
            '25g':5000,
            '100g':5000,
        }.get(x, 'error')    # 'error' is default if x not found

    def _amplitude_map(self, x): # checks the board type
        return {
            '2g':2,
            '4g':4,
            '10g':10,
            '20g':20,
            '25g':25,
            '100g':100,
        }.get(x, 'error')    # 'error' is default if x not found

    def _getSensitivity(self):
        '''Reads Modbus Registers 299-310 . Updates Trivibe Object Property
        info.internal_accelerometer_1["max_amplitude_gPK"]
                                    ,["max_frequency_hz"]
                                    ,["axis_1_sensitivity"]
                                    ,["axis_2_sensitivity"]
                                    ,["axis_3_sensitivity"]
        info.internal_accelerometer_2["max_amplitude_gPK"]
                                    ,["max_frequency_hz"]
                                    ,["axis_1_sensitivity"]
                                    ,["axis_2_sensitivity"]
                                    ,["axis_3_sensitivity"]'''
        s1_a1_sensitivity = self.modbus.read_float(299, functioncode=3, number_of_registers=2, byteorder=0)
        s1_a2_sensitivity = self.modbus.read_float(301, functioncode=3, number_of_registers=2, byteorder=0)
        s1_a3_sensitivity = self.modbus.read_float(303, functioncode=3, number_of_registers=2, byteorder=0)
        s2_a1_sensitivity = self.modbus.read_float(305, functioncode=3, number_of_registers=2, byteorder=0)
        s2_a2_sensitivity = self.modbus.read_float(307, functioncode=3, number_of_registers=2, byteorder=0)
        s2_a3_sensitivity = self.modbus.read_float(309, functioncode=3, number_of_registers=2, byteorder=0)
        self.info.setdefault("internal_accelerometer_1",{})
        self.info.get("internal_accelerometer_1").setdefault("max_amplitude_gPK", self._amplitude_map(self.info.get("board_type")[0]))
        self.info.get("internal_accelerometer_1").setdefault("max_frequency_hz", self._frequency_map(self.info.get("board_type")[0]))
        self.info.get("internal_accelerometer_1").setdefault("axis_1_sensitivity",s1_a1_sensitivity)
        self.info.get("internal_accelerometer_1").setdefault("axis_2_sensitivity",s1_a2_sensitivity)
        self.info.get("internal_accelerometer_1").setdefault("axis_3_sensitivity",s1_a3_sensitivity)
        self.info.setdefault("internal_accelerometer_2",{})
        self.info.get("internal_accelerometer_2").setdefault("max_amplitude_gPK", self._amplitude_map(self.info.get("board_type")[1]))
        self.info.get("internal_accelerometer_2").setdefault("max_frequency_hz", self._frequency_map(self.info.get("board_type")[1]))
        self.info.get("internal_accelerometer_2").setdefault("axis_1_sensitivity",s2_a1_sensitivity)
        self.info.get("internal_accelerometer_2").setdefault("axis_2_sensitivity",s2_a2_sensitivity)
        self.info.get("internal_accelerometer_2").setdefault("axis_3_sensitivity",s2_a3_sensitivity)
        time.sleep(0.1)

### GENERAL CONFIGURATION METHODS
    def reboot(self):
        self.modbus.write_register(1, 1)                             # Reboot Sensor
        time.sleep(6)

    def changeSlaveAddress(self, newSlaveAddress):
        UNLOCK = 45555
        LOCK = 45556
    
        if(newSlaveAddress < 2 or newSlaveAddress > 254):
            return('Invalid Slave Address: must be an integer >1 and <254')
        else:
            self.modbus.write_register(1, UNLOCK)                     # Unlock Control Register
            self.modbus.write_register(367, newSlaveAddress)         # Change Sensor Slave Address
            self.modbus.write_register(1, LOCK)                     # Lock Control Register
            self.reboot()
            self.modbus = minimalmodbus.Instrument(self.port, newSlaveAddress)    # Update Slave Address to New Address for our Tri-Vibe Object in minimal modbus

### OVERALL METHODS
    def updateTemperature(self):
        '''Reads Modbus Register 31. Scales to Celcius and Fahrenheit. Updates Trivibe Object Properties [temperature_C, temperature_F].'''
        temperature = self.modbus.read_register(31, functioncode=3)
        self.temperature = {
            "celcius":(temperature/10),
            "fahrenheit":((temperature/10*9/5) + 32)
        }
        try:
            self.temperature.update({
            "celcius":(temperature/10),
            "fahrenheit":((temperature/10*9/5) + 32)
        })
        except KeyError:
            self.temperature.setdefault({
            "celcius":(temperature/10),
            "fahrenheit":((temperature/10*9/5) + 32)
        })
        time.sleep(0.1)
        
    def updateUptime(self):
        '''Returns an Array with the [minutes,hours,days] a sensor has been running. If powered off or rebooted these values reset.'''
        sensor_uptime = self.modbus.read_registers(5, 3, functioncode=3)  # register_number, number_of_registers, function_code
        try:
            self.uptime.update({
            "minutes":sensor_uptime[0],
            "hours":sensor_uptime[1],
            "days":sensor_uptime[2]
        })
        except KeyError:
            self.uptime.setdefault({
            "minutes":sensor_uptime[0],
            "hours":sensor_uptime[1],
            "days":sensor_uptime[2]
        })
        time.sleep(0.1)

    def updateVibration(self):
        '''returns an array of overall vibration values:
        [
        axis_1_hybrid_accel, axis_2_hybrid_accel, axis_3_hybrid_accel,
        axis_1_hybrid_velocity, axis_2_hybrid_velocity, axis_3_hybrid_velocity,
        accelerometer_1_axis_1_accel, accelerometer_1_axis_2_accel, accelerometer_1_axis_3_accel,
        accelerometer_1_axis_1_velocity, accelerometer_1_axis_2_velocity, accelerometer_1_axis_3_velocity,
        accelerometer_2_axis_1_accel, accelerometer_2_axis_1_accel, accelerometer_3_axis_1_accel,
        accelerometer_2_axis_1_velocity, accelerometer_2_axis_2_velocity, accelerometer_2_axis_3_velocity
        ]'''
        hybrid_acceleration_a1 = self.modbus.read_float(172, functioncode=3, number_of_registers=2)
        hybrid_acceleration_a2 = self.modbus.read_float(174, functioncode=3, number_of_registers=2)
        hybrid_acceleration_a3 = self.modbus.read_float(176, functioncode=3, number_of_registers=2)
        hybrid_velocity_a1 = self.modbus.read_float(178, functioncode=3, number_of_registers=2)
        hybrid_velocity_a2 = self.modbus.read_float(180, functioncode=3, number_of_registers=2)
        hybrid_velocity_a3 = self.modbus.read_float(182, functioncode=3, number_of_registers=2)
#             accel_1_acceleration_a1 = self.modbus.read_float(190, functioncode=3, number_of_registers=2)
#             accel_1_acceleration_a2 = self.modbus.read_float(192, functioncode=3, number_of_registers=2)
#             accel_1_acceleration_a3 = self.modbus.read_float(194, functioncode=3, number_of_registers=2)
#             accel_1_velocity_a1 = self.modbus.read_float(196, functioncode=3, number_of_registers=2)
#             accel_1_velocity_a2 = self.modbus.read_float(198, functioncode=3, number_of_registers=2)
#             accel_1_velocity_a3 = self.modbus.read_float(200, functioncode=3, number_of_registers=2)
#             accel_2_acceleration_a1 = self.modbus.read_float(208, functioncode=3, number_of_registers=2)
#             accel_2_acceleration_a2 = self.modbus.read_float(210, functioncode=3, number_of_registers=2)
#             accel_2_acceleration_a3 = self.modbus.read_float(212, functioncode=3, number_of_registers=2)
#             accel_2_velocity_a1 = self.modbus.read_float(214, functioncode=3, number_of_registers=2)
#             accel_2_velocity_a2 = self.modbus.read_float(216, functioncode=3, number_of_registers=2)
#             accel_2_velocity_a3 = self.modbus.read_float(218, functioncode=3, number_of_registers=2)
        try:
            self.vibration.update({"axis_1_acceleration_g_RMS":hybrid_acceleration_a1,
                                    "axis_2_acceleration_g_RMS":hybrid_acceleration_a2,
                                    "axis_3_acceleration_g_RMS":hybrid_acceleration_a3,
                                    "axis_1_velocity_ips_RMS":hybrid_velocity_a1,
                                    "axis_2_velocity_ips_RMS":hybrid_velocity_a2,
                                    "axis_3_velocity_ips_RMS":hybrid_velocity_a3})
        except KeyError:
            self.vibration.setdefault({"axis_1_acceleration_g_RMS":hybrid_acceleration_a1,
                                    "axis_2_acceleration_g_RMS":hybrid_acceleration_a2,
                                    "axis_3_acceleration_g_RMS":hybrid_acceleration_a3,
                                    "axis_1_velocity_ips_RMS":hybrid_velocity_a1,
                                    "axis_2_velocity_ips_RMS":hybrid_velocity_a2,
                                    "axis_3_velocity_ips_RMS":hybrid_velocity_a3})
        time.sleep(0.1)


### ALARM METHODS

### ALARM CONFIGURATION METHODS
#         class ALARM:
#             '''There are 4 alarm channels that can be set in the TRIVIBE sensor. 
#                Typical Setup:
#                channel_1 = axis_1 acceleration or velocity
#                channel_2 = axis_2 acceleration or velocity
#                channel_3 = axis_3 acceleration or velocity
#                channel_4 = temperature'''
            
#             def __init__(self, channel):
#                 self.channel = channel
#                 self.axis = axis
#                 self.almtype = almtype
#                 self.lo = None
#                 self.hi = None
#                 self.hihi = None
#                 self.status = None
#                 self.hysteresis = None
#                 self.startAddress = self.getStartAddress()
#                 self.readSettings()
            
#             def getStartAddress(self):
#                 return {
#                         1:312,
#                         2:322,
#                         3:332,
#                         4:342,
#                 }.get(self.channel, 'error')
                
                
#             def readSettings(self):
#                 def type_check(x): # checks the type of alarm channel configuration
#                     return {
#                         0:'Unset',
#                         1:'Velocity',
#                         2:'Acceleration',
#                         3:'Displacement',
#                         4:'Temperature',
#                         5:'Impact Alert',
#                         6:'Danger Alert',
#                         7:'Detonation',
#                     }.get(x, 'error')    # 'error' is default if x not found

#                 def axis_check(x): # checks the axis of alarm channel configuration
#                     return{
#                         0:'Unset',
#                         1:'Axis 1',
#                         2:'Axis 2',
#                         3:'Axis 3',
#                         4:'Temperature',
#                         5:'Not Used',
#                         6:'Not Used',
#                         7:'Detonation Axis 1',
#                         8:'Detonation Axis 2',
#                         9:'Detonation Axis 3',
#                         10:'Impact Threshold Axis 1',
#                         11:'Impact Threshold Axis 2',
#                         12:'Impact Threshold Axis 3',
#                         13:'Impact Count Axis 1',
#                         14:'Impact Count Axis 2',
#                         15:'Impact Count Axis 3',
#                     }.get(x, 'error')
#                 try:
#                 alarm_axis = self.read_registers(312, 1, functioncode=3)
            
#             def getStatus(self):
#                 pass
            
#             def setLo(self):
#                 pass
            
#             def setHi(self):
#                 pass
            
#             def setHihi(self):
#                 pass

#             def getHighest(self):
#                 pass
            
#             def setTripDelay(self):
#                 pass
            
            


#             try:
#                 # channel 1 settings
#                 alarm_channel_1_axis = trivibe.read_registers(312, 1, functioncode=3)
#                 print("Alarm Channel 1 Axis: "+axis_check(alarm_channel_1_axis[0]))
#                 alarm_channel_1_type = trivibe.read_registers(313, 1, functioncode=3)
#                 print("Alarm Channel 1 Type: "+type_check(alarm_channel_1_type[0]))
#                 alarm_channel_1_low_limit = trivibe.read_float(314, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("Low: "+str(alarm_channel_1_low_limit))
#                 alarm_channel_1_high_limit = trivibe.read_float(316, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("High: "+str(alarm_channel_1_high_limit))
#                 alarm_channel_1_high_high_limit = trivibe.read_float(318, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("High-High: "+str(alarm_channel_1_high_high_limit))
#                 alarm_channel_1_hysteresis = trivibe.read_float(320, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("Hysteresis: "+str(alarm_channel_1_hysteresis))

#                 # channel 2 settings
#                 alarm_channel_2_axis = trivibe.read_registers(312, 1, functioncode=3)
#                 print("Alarm Channel 2 Axis: "+axis_check(alarm_channel_2_axis[0]))
#                 alarm_channel_2_type = trivibe.read_registers(313, 1, functioncode=3)
#                 print("Alarm Channel 2 Type: "+type_check(alarm_channel_2_type[0]))
#                 alarm_channel_2_low_limit = trivibe.read_float(314, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("Low: "+str(alarm_channel_2_low_limit))
#                 alarm_channel_2_high_limit = trivibe.read_float(316, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("High: "+str(alarm_channel_2_high_limit))
#                 alarm_channel_2_high_high_limit = trivibe.read_float(318, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("High-High: "+str(alarm_channel_2_high_high_limit))
#                 alarm_channel_2_hysteresis = trivibe.read_float(320, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("Hysteresis: "+str(alarm_channel_2_hysteresis))

#                 # channel 3 settings
#                 alarm_channel_3_axis = trivibe.read_registers(322, 1, functioncode=3)
#                 print("Alarm Channel 3 Axis: "+axis_check(alarm_channel_3_axis[0]))
#                 alarm_channel_3_type = trivibe.read_registers(323, 1, functioncode=3)
#                 print("Alarm Channel 3 Type: "+type_check(alarm_channel_3_type[0]))
#                 alarm_channel_3_low_limit = trivibe.read_float(324, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("Low: "+str(alarm_channel_3_low_limit))
#                 alarm_channel_3_high_limit = trivibe.read_float(326, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("High: "+str(alarm_channel_3_high_limit))
#                 alarm_channel_3_high_high_limit = trivibe.read_float(328, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("High-High: "+str(alarm_channel_3_high_high_limit))
#                 alarm_channel_3_hysteresis = trivibe.read_float(330, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("Hysteresis: "+str(alarm_channel_3_hysteresis))

#                 # channel 4 settings
#                 alarm_channel_4_axis = trivibe.read_registers(332, 1, functioncode=3)
#                 print("Alarm Channel 4 Axis: "+axis_check(alarm_channel_4_axis[0]))
#                 alarm_channel_4_type = trivibe.read_registers(333, 1, functioncode=3)
#                 print("Alarm Channel 4 Type: "+type_check(alarm_channel_4_type[0]))
#                 alarm_channel_4_low_limit = trivibe.read_float(334, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("Low: "+str(alarm_channel_4_low_limit))
#                 alarm_channel_4_high_limit = trivibe.read_float(336, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("High: "+str(alarm_channel_4_high_limit))
#                 alarm_channel_4_high_high_limit = trivibe.read_float(338, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("High-High: "+str(alarm_channel_4_high_high_limit))
#                 alarm_channel_4_hysteresis = trivibe.read_float(340, functioncode=3, number_of_registers=2, byteorder=0)
#                 print("Hysteresis: "+str(alarm_channel_4_hysteresis))

#             except IOError:
#                 return(self.readError)
        
#         def updateAlarmSettings(self):
#             pass

### SNAPSHOT METHODS
##########################################################################################################################################################################################
    def setSnapshotTime(self, milliseconds=1000):
        '''Set the length of time for the sensor to capture. Validates input: is 10ms increments between 10 & 30000.'''
        if(milliseconds % 10 == 0 and 10<milliseconds<30000):
            self.modbus.write_register(35, milliseconds, number_of_decimals=0, functioncode=16, signed=False)
            capture_time = self.modbus.read_register(35, functioncode=3)
            self.snapshot.setdefault("capture_time_ms", capture_time)
            self.snapshot.setdefault("capture_time_s", capture_time/1000)
        else:
            raise ValueError('milliseconds should be a 10ms increment between 10 and 30000.')
     
    def setSnapshotSamples(self, samples=16384):
        '''Set the how many samples per axis for the sensor to capture over the capture time. Valid inputs powers of 2 from 256 - 1048576.'''
        self.modbus.write_long(36, samples, signed=False, byteorder=0)
        samples_per_axis = self.modbus.read_long(36, functioncode=3, signed=False, byteorder=0)
        self.snapshot.setdefault("samples_per_axis", samples_per_axis)

    def _modbus_map_internal_accel(self, x):
        '''Accepts a value of 5 or 6 and returns a string for the internal accelerometer that was selected for snapshot collection.'''
        return {
            5:self.info["internal_accelerometer_1"],
            6:self.info["internal_accelerometer_2"],
        }.get(x, 'error')    # 'error' is default if x not found

    def setSnapshotAccel(self, internalAccel=5):
        '''Sets which of the two accelerometers inside the Tri-Vibe sensor to target when a snapshot is triggered. Valid Inputs: 5 or 6.'''
        self.modbus.write_register(32, internalAccel, number_of_decimals=0, functioncode=16, signed=False)
        selected_internal_accelerometer = self.modbus.read_register(32, functioncode=3)
        selected = self._modbus_map_internal_accel(selected_internal_accelerometer)
        try:
            self.snapshot.update({"internal_accelerometer":selected})
        except KeyError:
            self.snapshot.setdefault("internal_accelerometer", selected)

    def _modbus_map_capture_engine_status(self, x):
        return {
            0:"Idle",
            1:"Complete",
            2:"In Progress",
            3:"Start Error",
            4:"Timeout Error",
        }.get(x, 'error')    # 'error' is default if x not found

    def updateSnapshotEngineStatus(self):
        capture_engine_status = self.modbus.read_register(34, functioncode=3)
        time.sleep(0.1)
        try:
            self.snapshot.update({"capture_engine_status":self._modbus_map_capture_engine_status(capture_engine_status)})
        except KeyError:
            self.snapshot.setdefault("capture_engine_status", self._modbus_map_capture_engine_status(capture_engine_status))
    
    def triggerSnapshot(self):
        '''Trigger the sensor to begin collecting and storing triaxial RAW accelerometer data in sensor memory. Writes a value of 1 to register 33.'''
        self.modbus.write_register(33, 1, functioncode=16)
        try:
            self.snapshot.update({"timestamp":int(str(time.time())[slice(10)])})
        except KeyError:
            self.snapshot.setdefault("timestamp", int(str(time.time())[slice(10)]))
        try:
            self.snapshot.update({"datetime":str(datetime.fromtimestamp(self.snapshot.get("timestamp")))})
        except KeyError:
            self.snapshot.setdefault("datetime", str(datetime.fromtimestamp(self.snapshot.get("timestamp"))))
        try:
            self.snapshot.update({"triaxial_RAW_samples":[]})
        except KeyError:
            self.snapshot.setdefault("triaxial_RAW_samples",[])
        try:
            self.snapshot.update({"current_chunk":0})
        except KeyError:
            self.snapshot.setdefault("current_chunk",0)
        self._calcSnapshotTotalSamples()
        self._calcSnapshotChunks()
        self._createTimeArray()

    def _calcSnapshotTotalSamples(self):
        '''Gets the total number of samples held on the tri-vibe for the triaxial_raw data set.'''
        total_samples = self.snapshot.get("samples_per_axis")*3
        try:
            self.snapshot.update({"total_samples":total_samples})
        except KeyError:
            self.snapshot.setdefault("total_samples", total_samples)

    def _calcSnapshotChunks(self):
        '''The tri-vibe can return sets of 122 samples of the triaxial_raw  data set per read. This function calculates how many times we must read modbus registers 50 - 123 to get the complete data set.'''
        try:
            self.snapshot.update({"chunks_to_read":int(np.ceil(self.snapshot.get("total_samples")/122))})
        except KeyError:
            self.snapshot.setdefault("chunks_to_read", int(np.ceil(self.snapshot.get("total_samples")/122)))

    def _createTimeArray(self):
        '''Creates the x-axis of the timewave form chart.'''
        time_ms = self.snapshot.get("capture_time_ms")
        samples_per_axis = self.snapshot.get("samples_per_axis")
        step = time_ms/samples_per_axis
        if(step < 0.02):
            raise ValueError("Sample_Rate exceeds Tri-Vibe built in ADC capability of 50,000 samples per second. Please change capture_time or samples_per_axis so that time/samples >= 0.02")
        else:
            start = 0
            stop = time_ms
            time_array = np.arange(start, stop, step)
            try:
                self.snapshot.update({"time_array_ms":time_array})
            except KeyError:
                self.snapshot.setdefault("time_array_ms",time_array)

    def collectSnapshot(self):
        while(self.snapshot.get("current_chunk")<self.snapshot.get("chunks_to_read")):
                self.collectSnapshotChunk()
                self.snapshot["current_chunk"]+=1
        self._sliceToAxis()

    def collectSnapshotChunk(self):
        chunk_read = self.modbus.read_registers(49, 123, functioncode=3)
        try:
            self.snapshot.update({"start_sample_number_last_chunk_read":chunk_read[0]})
        except KeyError:
            self.snapshot.setdefault("start_sample_number_last_chunk_read", chunk_read[0])
        chunk_read.pop(0)
        for each in chunk_read:
            self.snapshot["triaxial_RAW_samples"].append(each)
        
    def createSnapshotXLSX(self):
        cwd = os.getcwd()                                # save Jupyter notebook's directory
        os.chdir('snapshots')                            # move to snapshots directory
        # Excel Sheet Storage of Data
        # Create a workbook and add a worksheet.
        workbook = xlsxwriter.Workbook(str(self.info.get("serial"))+'_'+str(self.snapshot.get("timestamp"))+'.xlsx')
        worksheet = workbook.add_worksheet()

        # Start from the first cell. Rows and columns are zero indexed.

        # Add Column Headers
        worksheet.write(0, 0, "timestamp")
        worksheet.write(0, 1, "time_ms")
        worksheet.write(0, 2, "a1_twf")
        worksheet.write(0, 3, "a2_twf")
        worksheet.write(0, 4, "a3_twf")
        worksheet.write(0, 5, "a1_sensitivity")
        worksheet.write(0, 6, "a2_sensitivity")
        worksheet.write(0, 7, "a3_sensitivity")
        worksheet.write(0, 8, "capture_time_ms")
        worksheet.write(0, 9, "samples_per_axis")

        row = 1
        col = 0
        # Iterate over timestamp data and write it out row by row.
        for each in self.snapshot.get("time_array_ms"):
            worksheet.write(row, col, self.snapshot.get("timestamp"))
            row += 1

        # Iterate over Amplitude data and write it out row by row.
        row = 1
        col = 1
        for each in self.snapshot.get("time_array_ms"):
            worksheet.write(row, col, each)
            row += 1

        row = 1
        col = 2
        for each in self.snapshot.get("axis_1_RAW"):
            worksheet.write(row, col, each)
            row += 1

        row = 1
        col = 3
        for each in self.snapshot.get("axis_2_RAW"):
            worksheet.write(row, col, each)
            row += 1

        row = 1
        col = 4
        for each in self.snapshot.get("axis_3_RAW"):
            worksheet.write(row, col, each)
            row += 1

        row = 1
        col = 5
        worksheet.write(row, col, self.snapshot.get("internal_accelerometer").get("axis_1_sensitivity"))

        row = 1
        col = 6
        worksheet.write(row, col, self.snapshot.get("internal_accelerometer").get("axis_2_sensitivity"))

        row = 1
        col = 7
        worksheet.write(row, col, self.snapshot.get("internal_accelerometer").get("axis_1_sensitivity"))

        row = 1
        col = 8
        worksheet.write(row, col, self.snapshot.get("capture_time_ms"))

        row = 1
        col = 9
        worksheet.write(row, col, self.snapshot.get("samples_per_axis"))

        workbook.close()
        os.chdir(cwd)
        print('Saved '+str(self.info.get("serial"))+'_'+str(self.snapshot.get("timestamp"))+'.xlsx''.xlsx File to /snapshots Directory')


    def _createFreqArray(self):
        n = np.size(self.snapshot.get("time_array")) #5000
        print(n)
        sample_rate = self.snapshot.get("sample_rate")
        print(sample_rate)
        freq_array = (sample_rate/2)*np.linspace(0,1,n//2)
        trimmed_frequency_array = freq_array
        self.snapshot.setdefault("freq_array",trimmed_frequency_array)

    def processCollectedSnapshot(self):
        if "axis_1_acceleration" not in self.snapshot:
            self.snapshot["axis_1_acceleration"]=[]
            self.snapshot["axis_2_acceleration"]=[]
            self.snapshot["axis_3_acceleration"]=[]
            self.processCollectedSnapshot()
        else:
            self._sliceToAxis()
            self.snapshot["axis_1_acceleration"] = self._processAxisAcceleration(self.snapshot["axis_1_RAW"],self.snapshot["internal_accelerometer"]["axis_1_sensitivity"])
            self.snapshot["axis_2_acceleration"] = self._processAxisAcceleration(self.snapshot["axis_2_RAW"],self.snapshot["internal_accelerometer"]["axis_2_sensitivity"])
            self.snapshot["axis_3_acceleration"] = self._processAxisAcceleration(self.snapshot["axis_3_RAW"],self.snapshot["internal_accelerometer"]["axis_3_sensitivity"])

    def _sliceToAxis(self):
        '''Takes the triaxial RAW Snapshot array and splits it into individual arrays of the individual axes. Note: We read 122 blocks of data at a time via Modbus to get the RAW samples... when sliced based on samples_per_axis the additional zeros sent in the last chunk read are ignored as expected.'''
        try:
            self.snapshot.update({"axis_1_RAW":self.snapshot["triaxial_RAW_samples"][0:self.snapshot["samples_per_axis"]],
                                  "axis_2_RAW":self.snapshot["triaxial_RAW_samples"][self.snapshot["samples_per_axis"]:(self.snapshot["samples_per_axis"]*2)],
                                  "axis_3_RAW":self.snapshot["triaxial_RAW_samples"][(self.snapshot["samples_per_axis"]*2):(self.snapshot["samples_per_axis"]*3)]})
        except KeyError:
            self.snapshot.setDefault({"axis_1_RAW":self.snapshot["triaxial_RAW_samples"][0:self.snapshot["samples_per_axis"]],
                                  "axis_2_RAW":self.snapshot["triaxial_RAW_samples"][self.snapshot["samples_per_axis"]:(self.snapshot["samples_per_axis"]*2)],
                                  "axis_3_RAW":self.snapshot["triaxial_RAW_samples"][(self.snapshot["samples_per_axis"]*2):(self.snapshot["samples_per_axis"]*3)]})

    
    def _processAxisAcceleration(self, array, sensitivity):
        '''Takes an axis array of RAW accelerometer data and converts to the Timewave Form values in Acceleration.'''
        new_array=[]
        constant_k = 3000/65535
        virtual_zero = self._getVirtualZero(min(array),max(array))
        for index in range(len(array)):
            if array[index] > virtual_zero:
                new_array.append((abs(array[index]-virtual_zero)*constant_k)/sensitivity)
            elif array[index] < virtual_zero:
                new_array.append((-abs(array[index]-virtual_zero)*constant_k)/sensitivity)
            else:
                new_array.append(0)
        return(new_array)
            
            
    def _getVirtualZero(self, minimum, maximum):
        '''Returns the zero position for an axis array.'''
        return((maximum-minimum)/2+minimum)
    
    def easySnapshot(self):
        self.setSnapshotTime()
        self.setSnapshotSamples()
        self.setSnapshotAccel()
        while(self.snapshot["capture_engine_status"]!="Idle"):
            self.updateSnapshotEngineStatus()
            time.sleep(0.5)
        self.triggerSnapshot()
        while(self.snapshot["capture_engine_status"]!="Complete"):
            self.updateSnapshotEngineStatus()
            time.sleep(0.5)
        self.collectSnapshot()
        self.processCollectedSnapshot()
    
        

<a id='notebooklibraries'></a>
---

# Part 3: Required Libraries for NOTEBOOK

In [3]:
# notebook libraries
import serial.tools.list_ports              # used to find COM Port of your USB to RS485 Converter

<a id='checkport'></a>
---

# Part 4: Find Computer's RS485 Port
#### Windows Users Port Address Should Look Like: 'COM3'
#### Linux Users Port Address Should Look Like: '/dev/ttyUSB0'
#### 1. You should pass the RS485 Port Address as the 2nd Argument when creating a sensor object.

In [4]:
def getPorts():
    ports = list(serial.tools.list_ports.comports())
    if len(ports)==0:
        raise Exception('You must plug in an RS485 to USB Converter or have an RS485 Port on your computer to run the rest of this notebook.')
    else:
        for p in ports:
            return(p)

print(getPorts())

COM5 - Standard Serial over Bluetooth link (COM5)


<a id='methodlist'></a>
---

# Part 5: Available TRIVIBE Methods (Functions)  
#### 1. Methods do one of the following things:  
* gather information from the sensor and store them in attributes.  
* change information already stored in attributes.  
* take information stored in attributes and store on the local filesystem  
* take information stored on the local filesystem and visualize it  

#### 2. The list of methods below are the methods built for users of of the TRIVIBE CLASS
#### 3. Implementation details can be read in the [TRIVIBE CLASS](#trivibeclass)  
#### 4. Each of the Methods will be shows in the [examples section](#examples)

In [5]:
method_list = [func for func in dir(TRIVIBE) if callable(getattr(TRIVIBE, func))and not func.startswith("__") and not func.startswith("_")]
for each in method_list:
    print(each+'()')

changeSlaveAddress()
collectSnapshot()
collectSnapshotChunk()
createSnapshotXLSX()
easySnapshot()
processCollectedSnapshot()
reboot()
setSnapshotAccel()
setSnapshotSamples()
setSnapshotTime()
triggerSnapshot()
updateSnapshotEngineStatus()
updateTemperature()
updateUptime()
updateVibration()


<a id='instantiation'></a>
---

# Part 6: Create a Sensor Object from [TRIVIBE CLASS](#trivibeclass)
#### 1. Required Arguments: MODBUS SLAVE ADDRESS and RS485 PORT ADDRESS.
#### 2. To reuse the Sensor Object throughout the project (NOTEBOOK), It must be assigned a variable name.
#### 3. Machine Saver prefers to use the Machine Name and Sensor's Position on the Machine as the variable name.  
#### 4. For examples taken from your own connected Tri-Vibe Sensor, simply edit the arguments below with your MODBUS SLAVE ADDRESS and RS485 PORT ADDRESS. Then re-run this notebook.

# TODO: Add Motor/Pump Picture with Sensors

In [6]:
motor_inboard = TRIVIBE(72, 'COM3')

<a id='attributelist'></a>
---

# Part 7: Available TRIVIBE Attributes (Properties)
#### 1. Attributes Store Information about a Tri-Vibe Sensor and its relevant data.

In [7]:
attribute_list = motor_inboard.__dict__
for each in attribute_list:
    print(each)

serialPort
modbus
info
uptime
vibration
temperature
alarms
snapshot


<a id='staticattributes'></a>
---

# Part 8: Static Attributes
#### 1. Some Attributes should not change throughout the lifecycle of the Tri-Vibe Sensor.
#### 2. The [Attributes](#attributelist) below are created when the Sensor Object is [created](#instantiation) and are not intended to change.

In [8]:
motor_inboard.info

{'serial': 41972,
 'revision': 109,
 'board_type': ['2g', '4g'],
 'internal_accelerometer_1': {'max_amplitude_gPK': 2,
  'max_frequency_hz': 1000,
  'axis_1_sensitivity': 661.4166870117188,
  'axis_2_sensitivity': 662.5479125976562,
  'axis_3_sensitivity': 658.4449462890625},
 'internal_accelerometer_2': {'max_amplitude_gPK': 4,
  'max_frequency_hz': 1000,
  'axis_1_sensitivity': 327.75286865234375,
  'axis_2_sensitivity': 332.95965576171875,
  'axis_3_sensitivity': 327.2760009765625}}

<a id='exampleuptime'></a>
---

# Part 9: Example Uptime
#### 1. Let's call the uptime [Attribute](#attributelist) for our example sensor.

In [9]:
motor_inboard.uptime

{}

#### 2. Notice how the uptime is an empty object within the sensor object
#### 3. This is because we have not yet called the Method which updates this Attribute
#### 4. Lets do that now using the updateUptime() [Method](#methodlist)

In [10]:
motor_inboard.updateUptime()

#### 5. Let's check the uptime [Attribute](#attributelist) after running the updateUptime() [Method](#methodlist)

In [11]:
motor_inboard.uptime

{'minutes': 5, 'hours': 1, 'days': 0}

#### 6. It is important to understand that the [Attributes](#attributelist) are NOT self updating (except for the [Static](#staticattributes) ones which are created when the Sensor Object is initialized).
#### 7. To guarantee up-to-date data, the appropriate [Methods](#methodlist) must be called to update the [Attribute](#attributelist) to be read for each Sensor Object BEFORE the Sensor Object's state is read.

<a id='exampletemperature'></a>
---

# Part 10: Example Temperature
#### 1. Call the updateTemperature() [Method](#methodlist)

In [12]:
motor_inboard.updateTemperature()

#### 2. Read the [Attribute](#attributelist) that was Updated

In [13]:
motor_inboard.temperature

{'celcius': 24.8, 'fahrenheit': 76.64}

<a id='examplevibration'></a>
---

# Part 11: Example Vibration
#### 1. Call the updateVibration() [Method](#methodlist)

In [14]:
motor_inboard.updateVibration()

#### 2. Read the [Attribute](#attributelist) that was Updated

In [15]:
motor_inboard.vibration

{'axis_1_acceleration_g_RMS': 0.01648396998643875,
 'axis_2_acceleration_g_RMS': 0.026615869253873825,
 'axis_3_acceleration_g_RMS': 0.05763695761561394,
 'axis_1_velocity_ips_RMS': 0.02998722530901432,
 'axis_2_velocity_ips_RMS': 0.022852154448628426,
 'axis_3_velocity_ips_RMS': 0.11021057516336441}

<a id='easysnapshot'></a>
---

## Part 12: Example Detailed Snapshot  
#### 1. Let's call the snapshot [Attribute](#attributelist) for our example sensor.

In [16]:
motor_inboard.snapshot

{}

#### 2. Notice how the snapshot object starts out empty within the sensor object.
#### 3. This is because we have not yet called the Methods which update this Attribute.
#### 4. Lets do that now using the easySnapshot() [Method](#methodlist).

# Setup Snapshot Parameters

In [17]:
motor_inboard.setSnapshotTime(2000)
motor_inboard.snapshot

{'capture_time_ms': 2000, 'capture_time_s': 2.0}

In [18]:
motor_inboard.setSnapshotSamples(16384)
motor_inboard.snapshot

{'capture_time_ms': 2000, 'capture_time_s': 2.0, 'samples_per_axis': 16384}

In [19]:
motor_inboard.setSnapshotAccel(5)
motor_inboard.snapshot.get("internal_accelerometer")

{'max_amplitude_gPK': 2,
 'max_frequency_hz': 1000,
 'axis_1_sensitivity': 661.4166870117188,
 'axis_2_sensitivity': 662.5479125976562,
 'axis_3_sensitivity': 658.4449462890625}

# Check Snapshot Engine is Idle before Triggering Snapshot

In [20]:
while(motor_inboard.snapshot.get("capture_engine_status")!="Idle"):
    motor_inboard.updateSnapshotEngineStatus()
    print(motor_inboard.snapshot.get("capture_engine_status"))
    time.sleep(0.5)
motor_inboard.triggerSnapshot()

Idle


# Check Snapshot Engine is Complete before Snapshot Collect

In [21]:
while(motor_inboard.snapshot.get("capture_engine_status")!= "Complete"):
    motor_inboard.updateSnapshotEngineStatus()
    print(motor_inboard.snapshot.get("capture_engine_status"))
    time.sleep(0.5)
motor_inboard.collectSnapshot()

In Progress
In Progress
In Progress
In Progress
Complete


# View Gathered Snapshot Object

In [22]:
# motor_inboard.snapshot.get("triaxial_RAW_samples")

The triaxial data set will likely have trailing zeroes because the total samples rarely fit cleanly into our 122 modbus register read.
So we need to slice the triaxial set up into individual axis using samples per axis which will ignore the padded zeroes at the end.

In [23]:
motor_inboard.snapshot.get("axis_1_RAW")
print(min(motor_inboard.snapshot.get("axis_1_RAW")))
print(max(motor_inboard.snapshot.get("axis_1_RAW")))
print(np.size(motor_inboard.snapshot.get("axis_1_RAW")))

34167
37889
16384


In [24]:
motor_inboard.snapshot.get("axis_2_RAW")
print(min(motor_inboard.snapshot.get("axis_2_RAW")))
print(max(motor_inboard.snapshot.get("axis_2_RAW")))
print(np.size(motor_inboard.snapshot.get("axis_2_RAW")))

18644
24107
16384


In [25]:
motor_inboard.snapshot.get("axis_3_RAW")
print(min(motor_inboard.snapshot.get("axis_3_RAW")))
print(max(motor_inboard.snapshot.get("axis_3_RAW")))
print(np.size(motor_inboard.snapshot.get("axis_3_RAW")))

33205
39065
16384


In [26]:
print(np.size(motor_inboard.snapshot.get("time_array_ms")))
print(max(motor_inboard.snapshot.get("time_array_ms")))

16384
1999.8779296875


Now that the minimal amount of data has been generated for the sensor, we can create an excel sheet with the data and save it and export it to the cloud (and process the FFT not using the precious resources of our embedded PC).

In [27]:
motor_inboard.createSnapshotXLSX()

Saved 41972_1587003952.xlsx.xlsx File to /snapshots Directory


### [Back to Top](#top)  